<a href="https://colab.research.google.com/github/zeynepylbrt/Task2_timeStamps/blob/main/Task2_timestamps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy python-dateutil dateparser
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!pip install isoweek  # For calendar week calculations


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 79.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# English Expressions Begin (Works Correct for most of the time expressions)

In [ ]:
from google.colab import files
import os
from datetime import datetime #creation_date used to create and manipulate dates.
import re #fmodule or regular expressions , used for pattern matching
import spacy # NLP library , it recognizes dates and times in text.
from dateutil.relativedelta import relativedelta #helps for the date calculations.
import dateparser #its for parsing the various date formats and expressions

# File Upload
uploaded = files.upload()
file_name = next(iter(uploaded))
print(f"Uploaded File: {file_name}")

with open(file_name, 'r') as file:
    text = file.read()

def word_to_number(word): # Converts words like "two" to numbers. Helps for the calculation of the actual date.
    """Convert word numbers and ordinals to integers."""
    word = word.lower().strip()

    numbers = {
        "first": 1, "second": 2, "third": 3, "fourth": 4, "fifth": 5,
        "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
        "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10,
        "1st": 1, "2nd": 2, "3rd": 3, "4th": 4, "5th": 5,
    }

    if word in numbers:
        return numbers[word]

    ordinal_match = re.match(r'(\d+)(?:st|nd|rd|th)', word)
    if ordinal_match:
        return int(ordinal_match.group(1))

    try:
        return int(word)
    except ValueError:
        return 0

def get_previous_weekday(date, weekday): # Calculate dates for exprssions like "Next Monday"
    """Calculate the previous occurrence of a given weekday."""
    days_behind = date.weekday() - weekday
    if days_behind <= 0:
        days_behind += 7
    return date - relativedelta(days=days_behind)

def get_next_weekday(date, weekday): # Same functionality with previous weekday but with "next"
    """Calculate the next occurrence of a given weekday."""
    days_ahead = weekday - date.weekday()
    if days_ahead <= 0:
        days_ahead += 7
    return date + relativedelta(days=days_ahead)

def get_nth_weekday_of_month(year, month, weekday, n):# Handles expressions like "first monday of January"
    """Calculate the nth occurrence of a weekday in a given month."""
    date = datetime(year, month, 1)

    # Find first occurrence using modulo arithmetic
    days_ahead = (weekday - date.weekday() + 7) % 7
    date += relativedelta(days=days_ahead)

    # Add weeks for nth occurrence
    result = date + relativedelta(weeks=n-1)

    if result.month != month:
        return None

    return result

def calculate_relative_dates(expr, reference_datetime):
    """Calculate dates from relative expressions."""
    days_of_week = {"monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
                    "friday": 4, "saturday": 5, "sunday": 6}
    months_of_year = {"january": 1, "february": 2, "march": 3, "april": 4,
                      "may": 5, "june": 6, "july": 7, "august": 8,
                      "september": 9, "october": 10, "november": 11, "december": 12}

    expr = expr.lower().strip()

    # Handle "tomorrow morning" and "yesterday morning"
    if expr in ["tomorrow morning", "yesterday morning"]:
        if expr.startswith("tomorrow"):
            date = reference_datetime + relativedelta(days=1)
        else:
            date = reference_datetime - relativedelta(days=1)
        return f"{expr} ({date.strftime('%d.%m.%Y')} morning)"

    # Handle "later" expressions
    later_match = re.match(r'(\d+|[a-z]+)\s*(days?|weeks?|months?|years?)\s+later', expr)
    if later_match:
        number, unit = later_match.groups()
        number = word_to_number(number)

        unit_mapping = {
            'day': 'days', 'days': 'days',
            'week': 'weeks', 'weeks': 'weeks',
            'month': 'months', 'months': 'months',
            'year': 'years', 'years': 'years'
        }

        if unit.lower() in unit_mapping and number > 0:
            future_date = reference_datetime + relativedelta(**{unit_mapping[unit.lower()]: number})
            return f"{expr} ({future_date.strftime('%d.%m.%Y')})"

    # Handle "in two months"
    if expr.startswith("in "):
        match = re.match(r'in\s+(\d+|[a-z]+)\s*(days?|weeks?|months?|years?)', expr)
        if match:
            number, unit = match.groups()
            number = word_to_number(number)

            unit_mapping = {
                'day': 'days', 'days': 'days',
                'week': 'weeks', 'weeks': 'weeks',
                'month': 'months', 'months': 'months',
                'year': 'years', 'years': 'years'
            }

            if unit.lower() in unit_mapping and number > 0:
                end_date = reference_datetime + relativedelta(**{unit_mapping[unit.lower()]: number})
                return f"{expr} ({end_date.strftime('%d.%m.%Y')})"

    # Handle "ago" expressions
    ago_match = re.match(r'(?:(\d+|[a-z]+)\s*(days?|weeks?|months?|years?)\s+ago)', expr)
    if ago_match:
        number, unit = ago_match.groups()
        number = word_to_number(number)

        unit_mapping = {
            'day': 'days', 'days': 'days',
            'week': 'weeks', 'weeks': 'weeks',
            'month': 'months', 'months': 'months',
            'year': 'years', 'years': 'years'
        }

        if unit.lower() in unit_mapping and number > 0:
            past_date = reference_datetime - relativedelta(**{unit_mapping[unit.lower()]: number})
            return f"{expr} ({past_date.strftime('%d.%m.%Y')})"

    # Handle "nth weekday of month" pattern
    nth_weekday_match = re.search(r'(?:the\s+)?(first|second|third|fourth|fifth|1st|2nd|3rd|4th|5th)\s+(\w+)\s+of\s+(\w+)', expr)
    if nth_weekday_match:
        nth, weekday, month = nth_weekday_match.groups()
        n = word_to_number(nth)
        weekday_num = days_of_week.get(weekday.lower())
        month_num = months_of_year.get(month.lower())

        if all([weekday_num is not None, month_num is not None, n > 0]):
            year = reference_datetime.year
            if month_num < reference_datetime.month:
                year += 1

            result_date = get_nth_weekday_of_month(year, month_num, weekday_num, n)
            if result_date:
                return f"{expr} ({result_date.strftime('%d.%m.%Y')})"

    # Handle "first day of month" pattern
    first_day_match = re.search(r'(?:the\s+)?(first|1st)\s+day\s+of\s+(\w+)(?:\s+(\d{4}))?', expr)
    if first_day_match:
        _, month, year = first_day_match.groups()
        month_num = months_of_year.get(month.lower())
        if month_num is not None:
            year = int(year) if year else reference_datetime.year
            if not year and month_num < reference_datetime.month:
                year = reference_datetime.year + 1
            result_date = datetime(year, month_num, 1)
            return f"{expr} ({result_date.strftime('%d.%m.%Y')})"

    # Handle relative time periods
    unit_mapping = {
        'day': 'days', 'days': 'days',
        'week': 'weeks', 'weeks': 'weeks',
        'month': 'months', 'months': 'months',
        'year': 'years', 'years': 'years'
    }

    match = re.match(r'(?:in\s+)?(\d+|[a-z]+)\s*(days?|weeks?|months?|years?)', expr)
    if match:
        number, unit = match.groups()
        number = word_to_number(number)

        if unit.lower() in unit_mapping and number > 0:
            has_in = expr.startswith('in ')
            formatted_expr = f"in {number} {unit}" if has_in else f"{number} {unit}"
            end_date = reference_datetime + relativedelta(**{unit_mapping[unit.lower()]: number})
            return f"{formatted_expr} ({reference_datetime.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')})"

    # Handle next/last weekday
    match = re.match(r'(next|last)\s+(\w+)', expr)
    if match:
        direction, day = match.groups()
        day_num = days_of_week.get(day)
        if day_num is not None:
            if direction == 'next':
                result = get_next_weekday(reference_datetime, day_num)
            else:
                result = get_previous_weekday(reference_datetime, day_num)
            return f"{expr} ({result.strftime('%d.%m.%Y')})"

    # Handle simple expressions
    if expr == 'tomorrow':
        result = reference_datetime + relativedelta(days=1)
        return f"{expr} ({result.strftime('%d.%m.%Y')})"
    elif expr == 'yesterday':
        result = reference_datetime - relativedelta(days=1)
        return f"{expr} ({result.strftime('%d.%m.%Y')})"
    elif expr == 'this evening':
        result = reference_datetime
        return f"{expr} ({result.strftime('%d.%m.%Y')})"

    # Fallback to dateparser
    try:
        parsed_date = dateparser.parse(expr, settings={'RELATIVE_BASE': reference_datetime})
        if parsed_date:
            return f"{expr} ({parsed_date.strftime('%d.%m.%Y')})"
    except:
        pass

    return None

# Main execution
creation_date = datetime.strptime('20.01.2025 17:34', '%d.%m.%Y %H:%M')  # Manually set for Colab
print(f"File Creation Date: {creation_date.strftime('%d.%m.%Y %H:%M:%S')}")

# SpaCy Model Loading
try:
    nlp = spacy.load("en_core_web_sm")
except Exception as e:
    print(f"spaCy model could not be loaded: {e}")
    nlp = None

# Text Processing / Uses spaCy (a natural language processing library) and regex to find time expressions in text
# looks for patterns matching dates and times
#extracts these expressions for processing

#if nlp:
#    doc = nlp(text)
#    time_expressions = []

#   for ent in doc.ents:
#        if ent.label_ in {"DATE", "TIME"}:
#            sent = next(s for s in doc.sents if ent.start >= s.start and ent.end <= s.end)
#            time_pattern = r'(?:in\s+(?:\d+|[a-z]+)\s*(?:days?|weeks?|months?|years?))|' \
#               r'(?:\d+|[a-z]+)\s*(?:days?|weeks?|months?|years?)\s*(?:ago)?|' \
#               r'(?:next|last)\s+\w+|' \
#               r'(?:tomorrow|yesterday)(?:\s+(?:morning|afternoon|evening|night))?|' \
#               r'(?:the\s+)?(?:first|second|third|fourth|fifth|1st|2nd|3rd|4th|5th)\s+\w+\s+of\s+\w+|' \
#               r'this\s+(?:morning|afternoon|evening|night)'

#            for match in re.finditer(time_pattern, sent.text.lower()):
#                if ent.start_char >= sent.start_char + match.start() and ent.end_char <= sent.start_char + match.end():
#                    original_text = text[sent.start_char + match.start():sent.start_char + match.end()]
#                    time_expressions.append(original_text)
#                    break
#else:
#    time_expressions = []

if nlp:
    doc = nlp(text)
    time_expressions = []

    # Define comprehensive time pattern
    time_pattern = r'''(?ix)                   # Case-insensitive and verbose mode
    (?:
        \b(?:in\s+(?:\d+|[a-z]+)\s*(?:days?|weeks?|months?|years?))|  # "in two months"
        (?:\d+|[a-z]+)\s*(?:days?|weeks?|months?|years?)\s*(?:ago|later)?|  # "two months ago" or "three days later"
        (?:next|last)\s+\w+|                                           # "next Monday"
        (?:tomorrow|yesterday)(?:\s+(?:morning|afternoon|evening|night))?|  # "tomorrow morning"
        (?:the\s+)?(?:first|second|third|fourth|fifth|1st|2nd|3rd|4th|5th)\s+\w+\s+of\s+\w+|  # "first Monday of January"
        this\s+(?:morning|afternoon|evening|night)                     # "this evening"
    )
    \b  # Word boundary
''' #Just for easier reading and maintain.

    # Two-step approach to find time expressions
    # 1. First, find all matches using regex
    matches = list(re.finditer(time_pattern, text, re.IGNORECASE))

    # 2. Then, verify with spaCy entities
    for match in matches:
        match_text = match.group()
        match_start = match.start()
        match_end = match.end()

        # Check if this span overlaps with any DATE or TIME entity
        for ent in doc.ents:
            if ent.label_ in {"DATE", "TIME"}:
                # Check for overlap between match and entity
                if (match_start <= ent.end_char and match_end >= ent.start_char):
                    time_expressions.append(match_text)
                    break

        # Also include expressions that match our pattern exactly
        # This helps catch phrases like "in two months" that SpaCy might miss
        if match_text.lower().strip() in [
            "in two months",
            "tomorrow morning",
            "yesterday morning",
            "this evening"
        ]:
            if match_text not in time_expressions:
                time_expressions.append(match_text)

else:
    time_expressions = []

# Analyze Time Expressions
reference_datetime = creation_date
print(f"\nReference Date: {reference_datetime.strftime('%d.%m.%Y %H:%M:%S')}")

for expr in time_expressions:
    calculated_date = calculate_relative_dates(expr, reference_datetime)
    if calculated_date:
        print(f"\nOriginal: {expr}")
        print(f"Corrected: {calculated_date}")
    else:
        print(f"\nOriginal: {expr}")
        print("Corrected: No valid date found.")

# Test case for "Yesterday morning"
#test_expr = "In two months"
#result = calculate_relative_dates(test_expr, reference_datetime)
#print(f"\nTest case:")
#print(f"Original: {test_expr}")
#print(f"Corrected: {result}")

Saving meetingNotes.txt to meetingNotes (8).txt
Uploaded File: meetingNotes (8).txt
File Creation Date: 20.01.2025 17:34:00

Reference Date: 20.01.2025 17:34:00

Original: the week 
Corrected: No valid date found.

Original: next Friday
Corrected: next friday (24.01.2025)

Original: Tomorrow
Corrected: tomorrow (21.01.2025)

Original: this evening
Corrected: this evening (20.01.2025)

Original: the first Monday of February
Corrected: the first monday of february (03.02.2025)

Original: Last Friday
Corrected: last friday (17.01.2025)

Original: in two weeks
Corrected: in two weeks (03.02.2025)

Original: ten days ago
Corrected: ten days ago (10.01.2025)

Original: Tomorrow morning
Corrected: tomorrow morning (21.01.2025 morning)

Original: In two months
Corrected: in two months (20.03.2025)

Original: Ten days ago
Corrected: ten days ago (10.01.2025)

Original: four months later
Corrected: four months later (20.05.2025)


In [ ]:
# German Expressions Begin (Works Mostly Correct for most of the time expressions)

In [ ]:
from google.colab import files
import os
from datetime import datetime
import re
import spacy
from dateutil.relativedelta import relativedelta
import dateparser
from datetime import datetime, timedelta
from dateutil.rrule import MO, TU, WE, TH, FR, SA, SU

# File Upload
uploaded = files.upload()
file_name = next(iter(uploaded))
print(f"Uploaded File: {file_name}")

with open(file_name, 'r', encoding='utf-8') as file:
    text = file.read()

# Configuration
reference_date = datetime.strptime('20.01.2025 17:34', '%d.%m.%Y %H:%M')
print(f"\nReference Date: {reference_date.strftime('%d.%m.%Y %H:%M:%S')}\n")

month_map = {
    "januar": 1, "februar": 2, "märz": 3, "april": 4, "mai": 5, "juni": 6,
    "juli": 7, "august": 8, "september": 9, "oktober": 10, "november": 11, "dezember": 12
}

season_map = {
    "frühling": (3, 5), "sommer": (6, 8), "herbst": (9, 11), "winter": (12, 2)
}

def parse_time_range(expr, base_date):
    """Handle time ranges and complex expressions."""
    expr_lower = expr.lower()

    # 1. Handle "morgen früh" with time window
    if "morgen früh" in expr_lower:
        tomorrow = base_date + relativedelta(days=1)
        return f"{tomorrow.strftime('%d.%m.%Y')} 07:00-10:00"

    # 2. Handle seasons with optional year
    season_year_match = re.match(r'(sommer|winter|frühling|herbst)\s*(\d{4})?', expr_lower)
    if season_year_match:
        season, year_str = season_year_match.groups()
        year = int(year_str) if year_str else base_date.year
        start_month, end_month = season_map[season]

        if start_month > end_month:  # Winter case
            return f"01.{start_month:02d}.{year} - 28.{end_month:02d}.{year+1}"
        return f"01.{start_month:02d}.{year} - 30.{end_month:02d}.{year}"

    return None

def parse_german_time(expr, base_date):
    """Main parser for German time expressions with enhanced features."""
    expr = expr.strip()
    expr_lower = expr.lower()

    # Preserve prefixes
    prefix = next((p for p in ['ab ', 'bis ', 'spätestens ', 'nach ', 'vor ']
                   if expr_lower.startswith(p)), '')
    if prefix:
        expr = expr[len(prefix):].strip()
        expr_lower = expr.lower()


    # Handle "ab morgen bis Sonntag"
    if ' bis ' in expr:
        parts = expr.split(' bis ', 1)
        start = parse_german_time(parts[0], base_date)
        end_date = parse_german_time(parts[1], base_date)

        # Handle Sunday calculation
        if "sonntag" in parts[1].lower():
            end_date = base_date + relativedelta(weekday=SU(+2))  # Next Sunday

        return f"{prefix}{start} - {end_date}"

    # Handle simple "morgen"
    if expr_lower == "morgen":
        tomorrow = base_date + relativedelta(days=1)
        return f"{prefix}{tomorrow.strftime('%d.%m.%Y')}"

    # 1. Handle "Mitte nächster Woche" (Fixed WE usage)
    if "mitte nächster woche" in expr_lower:
        next_week = base_date + relativedelta(weeks=1)
        middle_of_week = next_week + relativedelta(weekday=WE)  # Changed we -> WE
        return f"{prefix}Mitte {middle_of_week.strftime('%d.%m.%Y')}"

    # 2. Handle "übernächste Woche"
    if "übernächste woche" in expr_lower:
        date = base_date + relativedelta(weeks=2)
        return f"{prefix}{date.strftime('%d.%m.%Y')}"

    # 3. Handle quarters "erstes Quartal"
    quarter_match = re.match(r'(erstes|zweites|drittes|viertes)\s+quartal', expr_lower)
    if quarter_match:
        year = base_date.year
        quarters = {
            "erstes": (1, 1, 31, 3),
            "zweites": (4, 1, 30, 6),
            "drittes": (7, 1, 30, 9),
            "viertes": (10, 1, 31, 12)
        }
        q = quarter_match.group(1)
        start = datetime(year, quarters[q][0], quarters[q][1])
        end = datetime(year, quarters[q][3], quarters[q][2])
        return f"{start.strftime('%d.%m.%Y')} - {end.strftime('%d.%m.%Y')}"

    # 4. Handle "X Tage später"
    days_later_match = re.match(r'(\d+)\s+tage?\s+später', expr_lower)
    if days_later_match:
        days = int(days_later_match.group(1))
        result_date = base_date + relativedelta(days=days)
        return f"{prefix}{result_date.strftime('%d.%m.%Y')}"

    # 5. Handle calendar weeks "KW 13"
    kw_match = re.match(r'kw\s*(\d+)', expr_lower)
    if kw_match:
        week_num = int(kw_match.group(1))
        year = base_date.year
        date = datetime.fromisocalendar(year, week_num, 1)
        return f"{prefix}{date.strftime('%d.%m.%Y')}"

    # 6. Handle "nächste Woche Dienstag"
    next_week_day_match = re.match(r'(nächste woche|übernächste woche)\s+(\w+)', expr_lower)
    if next_week_day_match:
        weeks = 1 if "nächste" in next_week_day_match.group(1) else 2
        weekday = next_week_day_match.group(2)
        weekdays = ["montag", "dienstag", "mittwoch", "donnerstag",
                    "freitag", "samstag", "sonntag"]
        if weekday in weekdays:
            day_index = weekdays.index(weekday)
            date = base_date + relativedelta(weeks=weeks, weekday=day_index)
            return f"{prefix}{date.strftime('%d.%m.%Y')}"


    if time_range := parse_time_range(expr, base_date):
        return f"{prefix}{time_range}" if prefix else time_range

    # Rest of the existing dateparser logic
    settings = {'RELATIVE_BASE': base_date, 'PREFER_DATES_FROM': 'future'}
    parsed = dateparser.parse(expr_lower, languages=['de'], settings=settings)

    if parsed:
        # Handle time specifications
        if 'um' in expr_lower:
            time_match = re.search(r'um (\d{1,2}:\d{2})', expr)
            if time_match:
                parsed = parsed.replace(hour=int(time_match.group(1).split(':')[0]),
                                        minute=int(time_match.group(1).split(':')[1]))

        result = parsed.strftime('%d.%m.%Y %H:%M') if parsed.time() != datetime.min.time() else parsed.strftime('%d.%m.%Y')
        return f"{prefix}{result}" if prefix else result

    return f"{prefix}Kein valides Datum gefunden" if prefix else "Kein valides Datum gefunden"
def extract_time_expressions(text):
    """Improved extraction of German time expressions."""
    patterns = [
        r'\b(?:bis|ab|vor|nach|spätestens)\s+\d+\.?\s*\w+\b',
        r'\b\d+\s*(?:tage?|wochen?|monate?|jahre?)\s*(?:später|danach)\b',
        r'\b(?:nächste|übernächste|vorherige)\s+\w+\b',
        r'\b(?:anfang|mitte|ende)\s+(?:nächster?|dieser|übernächster?)\s*\w+\b',
        r'\b(?:erstes|zweites|drittes|viertes)\s+quartal\b',
        r'\bkw\s*\d+\b',
        r'\b\d{1,2}\.\s*(?:jan|feb|mär|apr|mai|jun|jul|aug|sep|okt|nov|dez)[a-z]*\b',
        r'\b(?:morgen|heute|gestern)\s+(?:früh|vormittag|nachmittag|abend)\b',
        r'\b(?:in|ab)\s+\d+\s+\w+\b',
        r'\b(?:sommer|winter|frühling|herbst)\b',
        r'\b\d{1,2}-\d{1,2}\s+uhr\b',
        r'\b(?:montag|dienstag|mittwoch|donnerstag|freitag|samstag|sonntag)\b'
    ]
    combined = re.compile('|'.join(patterns), flags=re.IGNORECASE)
    return sorted(set(match.group().strip() for match in re.finditer(combined, text)), key=len, reverse=True)

# Main processing
time_expressions = extract_time_expressions(text)

# Process and display results
for expr in time_expressions:
    parsed = parse_german_time(expr, reference_date)
    print(f"Original: {expr}")
    print(f"Parsed:   {parsed}\n{'-'*40}")

# Example test cases
test_cases = [
    "morgen früh",
    "nächste Woche Dienstag",
    "in 4 Wochen",
    "ab morgen bis Sonntag",
    "Sommer 2025",
    "2 Tage später",
    "Mitte nächster Woche",
    "morgen um 13:30 Uhr",
    "im neuen Jahr ab dem 9. Januar",
    "erstes Quartal 2025"
]

print("\nTEST CASES:")
for case in test_cases:
    parsed = parse_german_time(case, reference_date)
    print(f"Input:  {case}")
    print(f"Result: {parsed}\n{'-'*40}")

Saving time_expression_de.txt to time_expression_de (17).txt
Uploaded File: time_expression_de (17).txt

Reference Date: 20.01.2025 17:34:00

Original: Mitte nächster Woche
Parsed:   Mitte 29.01.2025
----------------------------------------
Original: übernächste Woche
Parsed:   03.02.2025
----------------------------------------
Original: erstes Quartal
Parsed:   01.01.2025 - 31.03.2025
----------------------------------------
Original: 2 Tage später
Parsed:   22.01.2025
----------------------------------------
Original: nächste Woche
Parsed:   27.01.2025 17:34
----------------------------------------
Original: 14. November
Parsed:   14.11.2025
----------------------------------------
Original: in 4 Wochen
Parsed:   17.02.2025 17:34
----------------------------------------
Original: morgen früh
Parsed:   21.01.2025 07:00-10:00
----------------------------------------
Original: 27. Januar
Parsed:   27.01.2025
----------------------------------------
Original: 6. Februar
Parsed:   06.02.